## Parallélisme à mémoire distribuée

### Les limitations de Python 

Dans cette partie nous allons voir comment se déroule en pratique le parallélisme à mémoire partagée, plus communément appelé multi-thread. 

Il faut cependant noter quelque chose d'assez cocace, c'est que Python n'est pas adapté pour faire du parallélisme à mémoire partagée, il est en fait incapable d'en faire à cause du GIL. C'est qui ce GIL ? C'est le Global Interpreter Lock et il porte bien son nom car il a pour seul vocation de bloquer l'execution paralléle de Python. 

Je suis sur que vous n'avez alors qu'une seule question qui vous vient à l'esprit, c'est pourquoi introduire ce GIL alors que toutes les machines disposent maintenant de processeur permettant de fairedu multi-threading ? La réponse le plus concise et la plus complète est : **c'est historique** bienvenu dans le monde du développement. Pour les curieux vous pouvez suivre ce [lien](https://wiki.python.org/moin/GlobalInterpreterLock) pour avoir plus d'informations. 



In [1]:
from threading import Thread
class PiComputerThreading(Thread):
    def __init__(self, nbpoint, start, end):
        super(PiComputerThreading, self).__init__()
        self._nbpoint = nbpoint
        self._start = start
        self._end = end
        self._res = 0.
        
    def run(self):
        self._internal()
        
    def _internal(self):
        s = 0
        l = 1./self._nbpoint
        for i in range(self._start, self._end):
            x = l * ( i + 0.5 )
            s += l * ( 4. / (1. + x**2 ) )
        self._res = s
        
    @property
    def pi(self):
        return self._res


In [4]:
def compute_pi_thread( nbpoint, nb_thread ):
    
    ## Split task 
    n = nbpoint // nb_thread
    offset = [ n*i for i in range(nb_thread+1)]
    offset[-1] += nbpoint % nb_thread
    
    runners = [ PiComputerThreading(nbpoint, offset[i], offset[i+1])  for i in range( nb_thread )  ]
        
    for t in runners:
        t.start()
        
    for t in runners:
        t.join()
        
    pi = 0;
    for t in runners:
        pi += t.pi
    return pi


In [12]:
from multiprocessing import Process
class PiComputerMultiProcess(Process):
    def __init__(self, nbpoint, start, end):
        super(PiComputerMultiProcess, self).__init__()
        self._nbpoint = nbpoint
        self._start = start
        self._end = end
        self._res = 0.
        
    def run(self):
        print("run")
        self._internal()
        
    def _internal(self):
        s = 0
        l = 1./self._nbpoint
        for i in range(self._start, self._end):
            x = l * ( i + 0.5 )
            s += l * ( 4. / (1. + x**2 ) )
        self._res = s
        print(s)
        
    @property
    def pi(self):
        return self._res

In [19]:
def compute_pi_multi( nbpoint, nb_thread ):
    
    ## Split task 
    n = nbpoint // nb_thread
    offset = [ n*i for i in range(nb_thread+1)]
    offset[-1] += nbpoint % nb_thread
    
    runners = [ PiComputerMultiProcess(nbpoint, offset[i], offset[i+1])  for i in range( nb_thread )  ]
        
    for t in runners:
        t.start()
        
    for t in runners:
        t.join()
        
    pi = 0;
    for t in runners:
        print(f"coucou {t.pi}")
        pi += t.pi
    return pi


In [20]:
pi_multi = compute_pi_multi(1000000, 2)


run
run
1.8545904360033396
1.287002217586562
> <ipython-input-19-9a57d74173be>(19)compute_pi_multi()
-> for t in runners:
(Pdb) n
> <ipython-input-19-9a57d74173be>(20)compute_pi_multi()
-> print(f"coucou {t.pi}")
(Pdb) p t
<PiComputerMultiProcess(PiComputerMultiProcess-13, stopped)>
(Pdb) p t._res
0.0
(Pdb) p t._start
0
(Pdb) p t._end
500000
(Pdb) q


BdbQuit: 

In [5]:
pi_thread = compute_pi_thread(1000000, 2)
pi_multi = compute_pi_multi(1000000, 2)

In [7]:
pi_thread

3.1415926535899015

In [8]:
pi_multi

0.0

In [ ]:
%timeit compute_pi_thread(10000000, 10)
%timeit compute_pi_multi(10000000, 2)